In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    RocCurveDisplay,
)
from sklearn.manifold import TSNE
import joblib

# === Chemins ===
model_dir = r"C:\Users\REINA\BRCA2_Pathogenicity\models"
viz_dir = r"C:\Users\REINA\BRCA2_Pathogenicity\visualizations"
os.makedirs(viz_dir, exist_ok=True)

# === Charger les données ===
df = pd.read_csv(r"C:\Users\REINA\BRCA2_Pathogenicity\data\processed\combined\brca2_combined_features_expanded.csv")

X = df.iloc[:, 2:].values
y = df["Label"].astype(int).values

# === Diviser ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.9, random_state=42
)

# === Normalisation ===
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# === Charger le modèle ===
clf = joblib.load(os.path.join(model_dir, "brca2_mlp_model_expanded.pkl"))

# === Prédictions ===
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

# === 1. Matrice de confusion ===
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Prédiction")
plt.ylabel("Vérité terrain")
plt.title("Matrice de confusion")
plt.savefig(os.path.join(viz_dir, "confusion_matrix.png"))
plt.close()

# === 2. Courbe ROC ===
roc_disp = RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("Courbe ROC")
plt.savefig(os.path.join(viz_dir, "roc_curve.png"))
plt.close()

# === 3. t-SNE ===
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_test_embedded = tsne.fit_transform(X_test)

plt.figure(figsize=(8,6))
palette = sns.color_palette("bright", 2)
sns.scatterplot(x=X_test_embedded[:,0], y=X_test_embedded[:,1], hue=y_test, palette=palette, legend="full")
plt.title("t-SNE des données test")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.legend(title="Label")
plt.savefig(os.path.join(viz_dir, "tSNE_plot.png"))
plt.close()

print(f"Visualisations sauvegardées dans {viz_dir}")


Visualisations sauvegardées dans C:\Users\REINA\BRCA2_Pathogenicity\visualizations
